# Solving Sudoku with Python

In [1]:
import pandas as pd
import numpy as np

In [2]:
#puzzles 

p1="""000003600
000760009
000092780
201050803
056030140
403070506
062480000
800015000
007900000"""

p2="""093850000
500073000
007400000
409010208
062040710
701030406
000005300
000120009
000096870"""

#hard puzzle
p3="""000304900
040065000
006800000
430000108
090000020
201000035
000006300
000210060
008507000"""

In [3]:
def is_correct(p):
    """Are all of the rows, cols, and boxes completed (1-9 entries)"""
    full = set(range(1,10))
    for c in range(0,9):
        for r in range(0,9): 
            for get_func in get_rect:
                if len(set(get_func(p,r,c)).intersection(full)) != 9:
                    return False
    return True



def is_complete(p):
    """Are all of the rows, cols, and boxes non zero"""
    for c in range(0,9):
        for r in range(0,9): 
            if p[r,c] == 0:
                return False                
    return True

In [4]:
def create_matrix():
    return np.zeros(dtype=int, shape=(9,9))

In [5]:
#convert strings to a matrix
def to_matrix(p):
    return np.array([np.array([int(i) for i in r]) for r in p.split('\n')])

In [10]:
# helper functions to pull associated rows, columns and boxes for a given cell.
get_row = lambda p,x,y: (p[x]).flatten()
get_col = lambda p,x,y: (p[:,y]).flatten()
get_box = lambda p,x,y: (p[3*int(x/3):3*int(x/3)+3,3*int(y/3):3*int(y/3)+3]).flatten()

get_rect = [get_row, get_col, get_box]

In [11]:
def get_possibilities(p):
    """For each entry in the grid, list out all of the things it could be"""
    full = set(range(1,10))
    possibiliteis = np.array([[set() for x in range(1, 10)] for x in range(1, 10)])
    for c in range(0,9):
        for r in range(0,9):
            candidates = set(full)
            if p[r,c] == 0:   
                for get_func in get_rect:
                    [candidates.remove(x) for x in set(get_func(p,r,c)) if x in candidates]
                possibiliteis[r,c] = candidates
            else:
                possibiliteis[r,c] = set([p[r,c]])
    return possibiliteis

def solve_step(p):
    """Attempt to solve the sudoku puzzle using 2 stratigies"""
    
    solution = create_matrix()

    is_update = False
    possibiliteis = get_possibilities(p)

    # if there is only one possible thing a grid item could be, then we have our answer!
    for c in range(0,9):
        for r in range(0,9):
            if p[r,c] == 0:   
                if len(possibiliteis[r,c]) == 1:
                    solution[r,c] = list(possibiliteis[r,c])[0]
                    is_update = True

    # This is sort of the opposite of the above
    # If this is the only cell in the row, col, box than can be a number - then make it so
    for c in range(0,9):
        for r in range(0,9):
            if p[r,c] == 0:    
                for get_func in get_rect:
                    #for each possibility, see if this is the only cell that can satisfy it
                    func_poss = [s for s in get_func(possibiliteis,r,c)]
                    func_poss.remove(possibiliteis[r,c])
                    poss = set(range(1,10))

                    for s in func_poss:
                        for e in s:
                            if e in poss:
                                poss.remove(e)
                    if len(poss) == 1:
                        solution[r,c] = poss.pop()
                        is_update = True
    if is_update:
        return solution
    else:
        return None


In [12]:
def solve(px, solution_list=None):

    if solution_list is None:
        solution_list = []
        
    p = px.copy()
    
    solution = solve_step(p)
    
    while solution is not None:
        p += solution
        solution_list.append(solution)
        solution = solve_step(p)
        
    if is_complete(p):
        if is_correct(p):
            return p, solution_list
        else:
            return None, None
    
    # find the first cell with multiple possibilities
    # make a guess!
    possibiliteis = get_possibilities(p)
    solution = create_matrix()
    
    min_pos = 11
    min_r = -1
    min_c = -1
    r_rows = np.arange(0, 9)
    c_rows = np.arange(0, 9)
    
    np.random.shuffle(r_rows)
    np.random.shuffle(c_rows)

    for c in c_rows:
        for r in r_rows:
            num_pos = len(possibiliteis[r,c])
            if num_pos > 1 and min_pos > num_pos:
                min_r = r
                min_c = c
                min_pos = num_pos
                
    if min_r == -1 and min_c==-1:
        return None, None
    
    poss_list = np.array(list(possibiliteis[min_r, min_c]))
    np.random.shuffle(poss_list)
    
    for poss in poss_list:
        p[min_r,min_c] = poss
        solution[min_r,min_c] = poss

        rs, slns = solve(p, solution_list + [solution])

        if rs is not None:
            if is_complete(rs):
                if is_correct(rs):
                    return rs, slns
        p[min_r,min_c] = 0
        solution[min_r,min_c] = 0
                               
    return None, None

In [13]:
p3_solution, steps = solve(to_matrix(p3))
print(p3_solution)
print(is_correct(p3_solution))
print(is_complete(p3_solution))

p1_solution, steps = solve(to_matrix(p1))
print(p1_solution)
print(is_correct(p1_solution))
print(is_complete(p1_solution))

p2_solution, slns = solve(to_matrix(p2))
print(p2_solution)
print(is_correct(p2_solution))
print(is_complete(p2_solution))

[[5 1 2 3 7 4 9 8 6]
 [8 4 3 9 6 5 2 7 1]
 [9 7 6 8 2 1 5 4 3]
 [4 3 7 6 5 2 1 9 8]
 [6 9 5 1 3 8 7 2 4]
 [2 8 1 7 4 9 6 3 5]
 [1 2 9 4 8 6 3 5 7]
 [7 5 4 2 1 3 8 6 9]
 [3 6 8 5 9 7 4 1 2]]
True
True
[[9 7 8 5 4 3 6 2 1]
 [1 2 4 7 6 8 3 5 9]
 [6 3 5 1 9 2 7 8 4]
 [2 9 1 6 5 4 8 7 3]
 [7 5 6 8 3 9 1 4 2]
 [4 8 3 2 7 1 5 9 6]
 [3 6 2 4 8 7 9 1 5]
 [8 4 9 3 1 5 2 6 7]
 [5 1 7 9 2 6 4 3 8]]
True
True
[[6 9 3 8 5 2 1 4 7]
 [5 1 4 9 7 3 6 8 2]
 [2 8 7 4 6 1 9 3 5]
 [4 3 9 6 1 7 2 5 8]
 [8 6 2 5 4 9 7 1 3]
 [7 5 1 2 3 8 4 9 6]
 [9 4 6 7 8 5 3 2 1]
 [3 7 8 1 2 4 5 6 9]
 [1 2 5 3 9 6 8 7 4]]
True
True


In [21]:
sln, steps = solve(create_matrix())
print(sln)
print(is_correct(sln))
print(is_complete(sln))
len(steps)

[[9 5 3 7 6 8 4 1 2]
 [2 8 4 5 1 3 9 7 6]
 [6 7 1 4 9 2 8 5 3]
 [3 9 7 6 8 1 2 4 5]
 [1 4 5 2 7 9 3 6 8]
 [8 6 2 3 5 4 7 9 1]
 [4 1 8 9 3 5 6 2 7]
 [7 3 9 1 2 6 5 8 4]
 [5 2 6 8 4 7 1 3 9]]
True
True


70

###### Huzzah!